# Build a Chatbot with Amazon Bedrock

## Objective One: Develop an Interactive Python Chat Application Utilizing Amazon Bedrock

In this notebook, we will  build a simple Chatbot that interacts with Amazon Bedrock, we'll use InMemoryChatMessageHistory to store the conversation history.
1. Install the required dependencies to interact with Amazon Bedrock
2. Configure parameters for the model to use
3. Initialize the chat model 
4. Invoke Bedrock
5. Perform a multi-turn conversation with the chatbot

## 1. Install the required dependencies to interact with Amazon Bedrock
Install the required packages needed. Ignore any pip dependency errors, they won't affect what we're doing.

In [ ]:
%pip install --upgrade -q botocore
%pip install --upgrade -q boto3
%pip install --upgrade -q awscli
%pip install --upgrade -q langchain_aws

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import boto3
import botocore

In [ ]:
from langchain_aws.chat_models import ChatBedrock
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

### Start a boto3 session
Needed to handle the session and manage interaction with AWS APIs

In [ ]:
boto3_session = boto3.session.Session()
region = boto3_session.region_name

## 2. Configure parameters for the model to use
Specify the foundation model, and we can also set up other parameters like temperature to control the level of creativity we want the model to use.

In [ ]:
model = "amazon.titan-text-lite-v1"
temperature = 0.1

## 3. Initialize the chat model 
Using the ChatBedrock class from LangChain, initialize a chat model that uses the Bedrock API. LangChain provides a framework to enable back and forth interactions between a user and model, combined with memory and using a chat interface.

In [ ]:
llm_chat = ChatBedrock(
    model_id=model, 
    model_kwargs={"temperature": temperature},
    region_name=region
)

### Configure conversation history
InMemoryChatMessageHistory stores messages in a memory list.
RunnableWithMessageHistory is used to handle message history during the conversation, saving each conversation with a session_id. It uses the session_id to identify up the relevant conversation history.

In [ ]:
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(llm_chat, 
                                                  get_session_history)


## 4. Invoke Bedrock 
Using LangChain to invoke Bedrock, maintaining the conversation history for our session, and providing a prompt.

In [ ]:
from langchain_core.messages import HumanMessage

response = with_message_history.invoke(
    [HumanMessage(content="Help me create a social post about 5 ways to stay productive")],
    config={"configurable": {"session_id": "1"}},
)
response.content

## 5. Perform a multi-turn conversation with the chatbot
Continue the conversation.

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Add one more tip that is completely different to the ones already provided")],
    config={"configurable": {"session_id": "1"}},
)
response.content

### Print the stored conversation history

In [ ]:
print(store)